In [3]:
%logstop
%logstart -ortq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [6]:
import gzip
import simplejson as json

In [7]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [8]:
scripts[:1]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [9]:
# scripts[:2]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [10]:
def describe(key):
    
    #     prepare list
    wl = []
    for i in scripts:
        wl.append(i[key])
    wl = sorted(wl)
    l = len(wl)

    
    #total
    total=0.0
    total = sum(wl)

    
    #avg
    avg = 0.0;
    avg = total/l
        
    
    #med
    med=0.0
    if not l % 2:
        # the condition executes if it evaluates as zero
        med = (wl[int(l/2)] + wl[int(l/2-1)])/2.0 #for even len
    else:
        med = wl[int(l/2)] # for odd len
        
        
    #s standard deviation
    s = 0.0
    v = 0.0
    for i in scripts:
        v += (i[key] - avg)**2
        s = (v/l)**0.5 # **0.5 - squareroot
    
    
    #quartiles
    if not l%2:
        q25 = wl[int(l*0.25)]
        q75 = wl[(int(l*0.75))]
    
    else:
        q25 = wl[int(l*0.25)]
        q75 = wl[(int(l*0.75)-1)]
    
    #print('wl=',wl)
    
    return (total, avg, s, q25, med, q75)
    

In [11]:
summary =  [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [12]:
# [('items', (4410054.0, 11.522744731217633, 35.05979472066032, 1, 3.0, 8)), ('quantity', (316356836.0, 826.5883059943667, 3959.423447612624, 30, 120.0, 466)), ('nic', (29048309.790000338, 75.89844899484315, 211.64970205862195, 7.7, 22.62, 65.94)), ('act_cost', (27053937.599999707, 70.68748295124895, 196.42716319851132, 7.25, 21.24, 61.53))]


In [13]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [14]:
bnf_names = [i['bnf_name'] for i in scripts]
bnf_names = list(set(bnf_names))
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [15]:
#refactored
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script['items'])
groupd = groups

In [16]:
max_item=['',0]
for k,v in groups.items():
    groups[k]=sum(v)

In [17]:
largest = max(groups.values())
for k,v in groups.items():
    if v == largest:
        max_item[1]=v
        max_item[0]=k

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [18]:
max_item=[tuple(max_item)]

In [108]:
            #2nd Method
# groups = {name: [] for name in bnf_names}
# for script in scripts:
#     groups[script['bnf_name']].append(script)
# largestItem =0
# for lis in groups.values():
#     total=0
#     for dic in lis:
#         for item in dic.items():
#             item = list(item)
#             if item[0]=='items':
#                 total+=item[1]
#     if total>largestItem:
#             largestItem=total
#             bnf_name = dic['bnf_name']
# max_item = [(bnf_name,largestItem)]
# print(max_item)

[('Omeprazole_Cap E/C 20mg', 113826)]


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [20]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


In [21]:
# adding new item to a tuple
# x=(3,4)
# x= x+(3, )
# for i in x:
#     print(i)

x=('Xismox XL 60_Tab 60mg',)
x=x[0].strip()
type(x)

str

**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [22]:
# #         method 1
# def group_by_field(data, *args):
#     field_list = []
#     for var in args:
#         field_list.append(var)
#     if len(field_list)>1:
#         keys = [(datum[field_list[0]], datum[field_list[1]]) for datum in data]
#     else:
#         keys = [(datum[field_list[0]]) for datum in data]
#     keys = list(set(keys))
#     groups = {key:[] for key in keys}             
#     return groups

In [23]:
#refactored and robust
def group_by_field2(data, *args, to_find=False):
    field_list = []
    for var in args:
        field_list.append(var)
    keys = {tuple(datum[lis] for lis in field_list) for datum in data}
    if len(field_list)==1:
        keys = {(key[0].strip()) for key in keys}
    groups = {key:[] for key in keys}
    if to_find and len(field_list)==1:
        for datum in data:
                groups[datum[field_list[0]]].append(datum[str(to_find)])
    elif not to_find and len(field_list)==1:
        for datum in data:
            groups[datum[field_list[0]]].append(datum)
    else:
        for datum in data:
            name = tuple(datum[field] for field in field_list)
            groups[name].append(datum)
    return groups

In [24]:
#test for more than one field
groupl=group_by_field2(scripts, 'bnf_name','bnf_code')

In [25]:
 # test for one field
groupl2=group_by_field2(scripts, 'bnf_name',to_find='items')

In [26]:
test_max_item=['',0]
for k,v in groupl2.items():
    groupl2[k]=sum(v)
largest = max(groupl2.values())

for k,v in groupl2.items():
    if v == largest:
        test_max_item[1]=v
        test_max_item[0]=k
test_max_item=[tuple(test_max_item)]

In [27]:
assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [28]:
practice_postal  = group_by_field2(practices, 'code', to_find='post_code')

In [29]:
for key, value in practice_postal.items():
    practice_postal[key]=min(value)

In [30]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [31]:
joined = scripts[:]

In [32]:
#adding a new key:value pair
for item in joined:
    item['postal_code']=practice_postal[item['practice']]

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [33]:
items_by_post = group_by_field2(joined, 'postal_code',to_find='items')

In [34]:
for k,v in items_by_post.items():
    items_by_post[k]=sum(v)

In [35]:
items_by_post=[(k,v) for k,v in items_by_post.items()]

In [145]:
items_by_post=sorted(items_by_post)
len(items_by_post)
# dict(items_by_post)

118

In [37]:
grader.score.pw__postal_totals(items_by_post[:100])

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

In [38]:
# practices[0]

In [39]:
joined[0] #scripts joined

{'bnf_code': '0101010G0AAABAB',
 'items': 2,
 'practice': 'N81013',
 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'nic': 5.98,
 'act_cost': 5.56,
 'quantity': 1000,
 'postal_code': 'SK11 6JL'}

In [40]:
# total_of_item = sum([i['items'] for i in joined])
# total_of_item

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [41]:
big_group = group_by_field2(joined, 'postal_code', 'bnf_name').items()
# list(big_group)[0]

In [42]:
total_items_by_bnf_post= {name: sum(group['items'] for group in groups) for name, groups in big_group}
total_items_by_bnf_post[('S65 1DA', 'Ticagrelor_Tab 90mg')]

39

In [43]:
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [44]:
total_items= [{'post_code': tup[0], 'bnf_name': tup[1], 'total' : tot} for tup, tot in list(total_items_by_bnf_post.items())]

In [45]:
total_items_by_post = group_by_field2(total_items, ('post_code'))
assert len(total_items_by_post) == 118

In [136]:
# len(total_items_by_post.values())

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [126]:
max_items_by_post = [sorted(group, key=lambda grip: (grip['total'], grip['bnf_name']), reverse=True)[0] 
                     for group in total_items_by_post.values()]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [137]:
result= [(item['post_code'], item['bnf_name'],item['total'] /dict(items_by_post)[item['post_code']]) 
         for item in max_items_by_post]

In [138]:

# Take the first 100 elements and pass it to the grader
items_by_region = sorted(result)[:100]
items_by_region

In [140]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*